# Story Text Regeneration Service Testing

This notebook tests the story generation API endpoints:
1. Generate Story Outline (Visual Story)
2. Generate Complete Story
3. Regenerate Story Segment


## Setup and Authentication


In [ ]:
import os
import json
import requests
import boto3
from pprint import pprint

# Set AWS environment
os.environ['AWS_PROFILE'] = 'sandbox'
os.environ['AWS_REGION'] = 'us-east-1'

# API Configuration
API_BASE_URL = 'https://6txczj8c6i.execute-api.us-east-1.amazonaws.com/dev'
CLIENT_ID = '1a09blthlvds6lq24afk6k2uuq'
USERNAME = 'admin@example.com'
PASSWORD = 'NewPassword123!'

print("✓ Configuration loaded")


✓ Configuration loaded


In [ ]:
# Get authentication token
print("🔐 Getting authentication token...")

cognito_client = boto3.client('cognito-idp', region_name='us-east-1')

auth_response = cognito_client.initiate_auth(
    ClientId=CLIENT_ID,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': USERNAME,
        'PASSWORD': PASSWORD
    }
)

TOKEN = auth_response['AuthenticationResult']['IdToken']
print(f"✓ Token obtained ({len(TOKEN)} characters)")

# Prepare headers for API calls
headers = {
    'Authorization': TOKEN,
    'Content-Type': 'application/json'
}


🔐 Getting authentication token...
✓ Token obtained (1118 characters)


## Test 1: Generate Story Outline (Visual Story)

Generate a story outline for a visual story about a brave mouse named Max.


In [ ]:
print("📋 TEST 1: Generate Story Outline (Visual Story)")
print("─" * 70)

outline_payload = {
    "genre": "adventure",
    "reading_level": "elementary",
    "tone": "lighthearted",
    "user_input_description": "A brave little mouse named Max who discovers a magical cheese that grants wishes",
    "story_type": "visual",
    "number_of_speakers": 3,
    "panels": 6,
    "job_id": "test-job-outline-001"
}

print("\nRequest payload:")
pprint(outline_payload)

print("\nSending request...")
response = requests.post(
    f"{API_BASE_URL}/generate-story-outline",
    headers=headers,
    json=outline_payload
)

print(f"\nResponse Status: {response.status_code}")
print("\nResponse Body:")
outline_response = response.json()
pprint(outline_response)

if response.status_code == 200:
    print("\n✅ PASS - Story outline generation successful")
    story_parts = outline_response['result']['story_parts']
    print("\n📝 Story parts extracted for next test")
    
    # Export response to JSON file
    output_file = "test1_outline_response.json"
    with open(output_file, 'w') as f:
        json.dump(outline_response, f, indent=2)
    print(f"💾 Response exported to: {output_file}")
else:
    print(f"\n❌ FAIL - Expected 200 but got {response.status_code}")
    raise Exception("Test 1 failed")


📋 TEST 1: Generate Story Outline (Visual Story)
──────────────────────────────────────────────────────────────────────

Request payload:
{'genre': 'adventure',
 'job_id': 'test-job-outline-001',
 'number_of_speakers': 3,
 'panels': 6,
 'reading_level': 'elementary',
 'story_type': 'visual',
 'tone': 'lighthearted',
 'user_input_description': 'A brave little mouse named Max who discovers a '
                           'magical cheese that grants wishes'}

Sending request...

Response Status: 200

Response Body:
{'authenticated_user': {'email': 'admin@example.com',
                        'sub': '3488e408-5061-702b-2c5d-83be4c54f069',
                        'username': '3488e408-5061-702b-2c5d-83be4c54f069'},
 'request_id': '6dde8e33-556b-42a7-8846-f1d1e180b2a6',
 'result': {'metadata': {'created_timestamp': '2025-10-03T15:49:39.471427+00:00',
                         'genre': 'adventure',
                         'job_id': 'test-job-outline-001',
                         'number_of_spe

## Test 2: Generate Complete Story

Generate a complete story using the outline from Test 1.


In [ ]:
print("📋 TEST 2: Generate Complete Story (Visual, 6 panels)")
print("─" * 70)

story_payload = {
    "genre": "adventure",
    "reading_level": "elementary",
    "tone": "lighthearted",
    "story_outline_description": story_parts,
    "story_type": "visual",
    "number_of_speakers": 3,
    "panels": 6,
    "job_id": "test-job-story-002"
}

print("\nRequest payload (story_outline_description from previous test):")
pprint(story_payload)

print("\nSending request...")
response = requests.post(
    f"{API_BASE_URL}/generate-story",
    headers=headers,
    json=story_payload
)

print(f"\nResponse Status: {response.status_code}")
print("\nResponse Body:")
story_response = response.json()
pprint(story_response)

if response.status_code == 200:
    print("\n✅ PASS - Complete story generation successful")
    
    # Extract segments for regeneration test
    segments = []
    for story_part in story_response['result']['story_parts']:
        for section in story_part['sections']:
            for segment in section['segments']:
                segments.append({
                    'segment_num': segment['segment_num'],
                    'segment_content': segment['segment_content'],
                    'speaker': segment['speaker']
                })
    
    # Pick the 3rd segment for regeneration (index 2)
    segment_to_regen = segments[2]
    segment_num = segment_to_regen['segment_num']
    
    print(f"\n📝 Extracted segment #{segment_num} for regeneration test")
    
    # Export response to JSON file
    output_file = "test2_story_response.json"
    with open(output_file, 'w') as f:
        json.dump(story_response, f, indent=2)
    print(f"💾 Response exported to: {output_file}")
else:
    print(f"\n❌ FAIL - Expected 200 but got {response.status_code}")
    raise Exception("Test 2 failed")


📋 TEST 2: Generate Complete Story (Visual, 6 panels)
──────────────────────────────────────────────────────────────────────

Request payload (story_outline_description from previous test):
{'genre': 'adventure',
 'job_id': 'test-job-story-002',
 'number_of_speakers': 3,
 'panels': 6,
 'reading_level': 'elementary',
 'story_outline_description': [{'story_part': 'beginning',
                                'story_part_speakers': ['Narrator',
                                                        'Max the Mouse',
                                                        'Glimmer the Cheese '
                                                        'Fairy'],
                                'story_part_summary': 'In a cozy mouse hole '
                                                      'beneath an old bakery, '
                                                      'Max the Mouse lives a '
                                                      'simple life collecting '
                      

## Test 3: Regenerate Story Segment

Regenerate a specific segment with a user request to make it more exciting.


In [ ]:
print(f"📋 TEST 3: Regenerate Story Segment #{segment_num}")
print("─" * 70)

regenerate_payload = {
    "job_id": "test-job-regen-003",
    "user_request": "Make this segment more exciting with additional action",
    "original_story_segments": segments,
    "original_story_segment_num": segment_num,
    "original_story_segment": segment_to_regen,
    "genre": "adventure",
    "reading_level": "elementary",
    "tone": "lighthearted",
    "story_type": "visual",
    "number_of_speakers": 3,
    "panels": 6
}

print("\nRequest payload:")
pprint(regenerate_payload)

print(f"\nSending request to regenerate segment {segment_num}...")
response = requests.post(
    f"{API_BASE_URL}/regenerate-segment",
    headers=headers,
    json=regenerate_payload
)

print(f"\nResponse Status: {response.status_code}")
print("\nResponse Body:")
regenerate_response = response.json()
pprint(regenerate_response)

if response.status_code == 200:
    print("\n✅ PASS - Segment regeneration successful")
    
    # Show before and after comparison
    original = segment_to_regen['segment_content']
    new = regenerate_response['result']['new_story_segment']['segment_content']
    
    print("\n📊 Comparison:")
    print(f"  Original: {original}")
    print(f"  Regenerated: {new}")
    
    # Export response to JSON file
    output_file = "test3_regenerate_response.json"
    with open(output_file, 'w') as f:
        json.dump(regenerate_response, f, indent=2)
    print(f"💾 Response exported to: {output_file}")
else:
    print(f"\n❌ FAIL - Expected 200 but got {response.status_code}")
    raise Exception("Test 3 failed")


📋 TEST 3: Regenerate Story Segment #3
──────────────────────────────────────────────────────────────────────

Request payload:
{'genre': 'adventure',
 'job_id': 'test-job-regen-003',
 'number_of_speakers': 3,
 'original_story_segment': {'segment_content': "What could that be? It's so "
                                               'sparkly!',
                            'segment_num': 3,
                            'speaker': 'Max the Mouse'},
 'original_story_segment_num': 3,
 'original_story_segments': [{'segment_content': 'In a cozy mouse hole beneath '
                                                 'an old bakery lived Max the '
                                                 'Mouse, who spent his days '
                                                 'collecting crumbs and '
                                                 'dreaming of adventure.',
                              'segment_num': 1,
                              'speaker': 'Narrator'},
                           

## Summary


In [ ]:
print("═" * 70)
print("🎭 All 3 tests completed successfully!")
print("═" * 70)


══════════════════════════════════════════════════════════════════════
🎭 All 3 tests completed successfully!
══════════════════════════════════════════════════════════════════════
